In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp

In [ ]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import *
from sparknlp.annotator import *
from sparknlp.base import *

In [ ]:
spark = sparknlp.start()

In [ ]:
df = pd.read_csv('processed_data_Ukraine.csv')

In [ ]:
df = df[['content']]
df['content'] = df['content'].astype(str)
df = df[df['content'].str.split().str.len().lt(30) & df['content'].str.split().str.len().gt(6)]
df.reset_index(inplace=True)
df.head()

In [ ]:
from pyspark.sql import SparkSession
sparkdf = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()
data = sparkdf.createDataFrame(df)

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("content")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name='classifierdl_use_emotion')\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

In [ ]:
emo_labels = { emo:(i+1) for i, emo in enumerate("surprise joy sadness fear".split())}

def emotion_processing(data, pipeline):
  processed_tweets = pipeline.fit(data.select('content')).transform(data.select('content'))
  text = list((((processed_tweets.select(F.explode(F.arrays_zip('document.result')).alias("col")).select(F.expr("col").alias("text"))).toPandas())).text)
  text = [text[0] for text in text] # extract raw text
  score = list((((processed_tweets.select(F.explode(F.arrays_zip('sentiment.result')).alias("col1")).select(F.expr("col1").alias("sentiment"))).toPandas())).sentiment)
  
  df1 = pd.DataFrame(list(zip(text,score)), columns = ['content','sentiment'])
  df1['emotion'] = df1['sentiment'].apply(lambda x: x[0])
  df1['label'] = df1['emotion'].apply(lambda x: emo_labels[x])
  
  df1.drop(['sentiment'], axis=1, inplace=True)
  return df1



In [ ]:
df_new = emotion_processing(data, nlpPipeline)

In [ ]:
df_new.to_csv('sparknlp_Ukraine.csv')